In [1]:
import polars as pl
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
import torch.optim as optim

import math
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars

ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed-label
f64,str,str,i32,str,i32,i64,i64,f64,f64,f64,i64,str,str,i32,i64,i64,i64,i64,i64,str,i32,str
1.5322e9,"""C63mNb4FhdpSoHAj9g""","""192.168.100.108""",5526,"""47.138.157.173""",37215,0,0,0.0,0.0,0.0,0,"""-""","""-""",0,0,1,40,0,0,"""(empty)""",1,"""Okiru"""
1.5326e9,"""CppkaidEFlUgsYbOh""","""192.168.100.111""",60403,"""147.32.6.210""",23,1,0,0.0,0.0,0.0,2,"""-""","""-""",0,7,1,40,0,0,"""(empty)""",1,"""PartOfAHorizontalPortScan"""
1.5326e9,"""CNXYoV2KOSjeK0XsCi""","""192.168.100.111""",13386,"""50.229.102.190""",81,1,0,0.0,0.0,0.0,2,"""-""","""-""",0,7,1,40,0,0,"""(empty)""",1,"""PartOfAHorizontalPortScan"""
1.5455e9,"""Cv91cy38cVUjBeny4k""","""192.168.1.198""",36097,"""78.87.208.14""",37215,0,0,null,null,null,0,"""-""","""-""",0,0,1,40,0,0,"""-""",1,"""Okiru"""
1.5454e9,"""CKvVZA3voOeDO627Ca""","""192.168.1.198""",36097,"""202.99.70.13""",37215,0,0,null,null,null,0,"""-""","""-""",0,0,1,40,0,0,"""-""",1,"""Okiru"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1.5514e9,"""CwwQzn3658DIsMZ9uj""","""192.168.1.193""",30535,"""217.130.2.16""",8081,0,1,0.000005,0.0,0.0,1,"""-""","""-""",0,1,2,80,0,0,"""-""",1,"""PartOfAHorizontalPortScan"""
1.5454e9,"""CnykMt2RDdLvT42t8l""","""192.168.1.198""",36097,"""112.160.204.32""",37215,0,0,null,null,null,0,"""-""","""-""",0,0,1,40,0,0,"""-""",1,"""Okiru"""
1.5514e9,"""CDjM7C2ZIehXyoB3f""","""192.168.1.200""",41258,"""134.130.151.164""",23,0,1,0.000002,0.0,0.0,1,"""-""","""-""",0,1,2,120,0,0,"""-""",1,"""PartOfAHorizontalPortScan"""


In [7]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [8]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [9]:
df_polars = df_polars.drop_nulls()

In [10]:
X = df_polars.drop('label')
y = df_polars['label']       

In [11]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

In [12]:
# calcular número desejado de características
def calculate_desired_num_features(current_num_features):
    root = math.sqrt(current_num_features)
    desired_num_features = math.ceil(root) ** 2
    
    return desired_num_features

In [13]:
print(X.shape)


(555233, 5)


# Treinamento

In [14]:
results = []
SAMPLE_2D_SIZE = 4

In [15]:
class CNNModel(nn.Module):
    def __init__(self, conv1_out_channels=64, conv2_out_channels=64, dropout_rate=0.2):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(1, conv1_out_channels, kernel_size=2, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(conv1_out_channels, conv2_out_channels, kernel_size=2, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        def conv2d_out_size(size, kernel_size=2, stride=1, padding=0):
            return (size - kernel_size + 2 * padding) // stride + 1
        
        size_after_conv1 = conv2d_out_size(SAMPLE_2D_SIZE, kernel_size=2)
        size_after_conv2 = conv2d_out_size(size_after_conv1, kernel_size=2)
        size_after_pool = conv2d_out_size(size_after_conv2, kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(conv2_out_channels * size_after_pool * size_after_pool, 64)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x.squeeze()


In [16]:
def startTrain(conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.2, learning_rate=0.001, num_epochs=10):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    results_fold = []
    
    for train_idx, test_idx in kfold.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
    
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
    
        desired_num_features = 16
        current_num_features = X_train_scaled.shape[1]
    
        if current_num_features < desired_num_features:
            padding = desired_num_features - current_num_features
            X_train_scaled = np.pad(X_train_scaled, ((0, 0), (0, padding)), 'constant')
            X_test_scaled = np.pad(X_test_scaled, ((0, 0), (0, padding)), 'constant')
        elif current_num_features > desired_num_features:
            raise ValueError("Número de características maior que o desejado.")
    
        X_train_scaled = X_train_scaled.reshape(-1, 1, SAMPLE_2D_SIZE, SAMPLE_2D_SIZE)
        X_test_scaled = X_test_scaled.reshape(-1, 1, SAMPLE_2D_SIZE, SAMPLE_2D_SIZE)
    
        X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
    
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = CNNModel(conv1_out_channels=conv1_out_channels, conv2_out_channels=conv2_out_channels, dropout_rate=dropout_rate)
        
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
        batch_size = 512
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)
        
        for epoch in range(num_epochs):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
    
        model.eval()
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                preds = torch.sigmoid(outputs)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(y_batch.cpu().numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        y_pred = (all_preds >= 0.5).astype(int)
        y_true = all_labels
    
        accuracy = accuracy_score(y_true, y_pred)
        results_fold.append(accuracy)
    return np.mean(results_fold)

In [17]:
def grid_search():
    param_grid = {
        'conv1_out_channels': [32, 64],
        'conv2_out_channels': [32, 64],
        'dropout_rate': [0.2, 0.3, 0.4],
        'learning_rate': [0.001, 0.0001],
        'num_epochs': [6, 10]
    }

    best_accuracy = 0
    best_params = {}

    for conv1_out_channels in param_grid['conv1_out_channels']:
        for conv2_out_channels in param_grid['conv2_out_channels']:
            for dropout_rate in param_grid['dropout_rate']:
                for learning_rate in param_grid['learning_rate']:
                    for num_epochs in param_grid['num_epochs']:
                        accuracy = startTrain(conv1_out_channels, conv2_out_channels, dropout_rate, learning_rate, num_epochs)
                        print(f"conv1_out_channels={conv1_out_channels}, conv2_out_channels={conv2_out_channels}, "
                              f"dropout_rate={dropout_rate}, learning_rate={learning_rate}, num_epochs={num_epochs} "
                              f"-> Accuracy: {accuracy}")

                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params = {
                                'conv1_out_channels': conv1_out_channels,
                                'conv2_out_channels': conv2_out_channels,
                                'dropout_rate': dropout_rate,
                                'learning_rate': learning_rate,
                                'num_epochs': num_epochs
                            }

    print("\nMelhores Hiperparâmetros encontrados:", best_params)
    print(f"Melhor Acurácia Média: {best_accuracy}")
    return best_params


In [18]:
inicio = time.time()
best_params = grid_search()
fim = time.time()

conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.2, learning_rate=0.001, num_epochs=6 -> Accuracy: 0.9884931197956517
conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.2, learning_rate=0.001, num_epochs=10 -> Accuracy: 0.9901392717325541
conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.2, learning_rate=0.0001, num_epochs=6 -> Accuracy: 0.9815861084500692
conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.2, learning_rate=0.0001, num_epochs=10 -> Accuracy: 0.981326758951294
conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.3, learning_rate=0.001, num_epochs=6 -> Accuracy: 0.9881221004656207
conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.3, learning_rate=0.001, num_epochs=10 -> Accuracy: 0.9896187708199855
conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.3, learning_rate=0.0001, num_epochs=6 -> Accuracy: 0.9812997414819122
conv1_out_channels=32, conv2_out_channels=32, dropout_rate=0.3, learning_rate=0.0001,

In [19]:
duracao = fim - inicio
print(duracao)

18475.200475931168


In [20]:
print("\nMelhores Hiperparâmetros encontrados:", best_params)


Melhores Hiperparâmetros encontrados: {'conv1_out_channels': 64, 'conv2_out_channels': 64, 'dropout_rate': 0.3, 'learning_rate': 0.001, 'num_epochs': 10}
